This Notebook is used for capstone project

In [2]:

import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install geopy
!conda install -c conda-forge lxml --yes
print('Packages installed.')

import pandas as pd
import numpy as np

import geocoder # import geocoder
import requests
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


     |████████████████████████████████| 102kB 8.5MB/s ta 0:00:011
     |████████████████████████████████| 829kB 23.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 112kB 5.0MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxml2-2.9.10             |

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(link,header=0)
df = tables[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
# set the column names
df.columns = ['PostalCode', 'Borough', 'Neighborhood']


# apply condition as if borough is not assigned then ignore that row and if Neighthood is not assigned then add the value of Borough to the Neighborhood
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

    if row['Borough'] == 'Not assigned':
        df.drop([index], axis=0, inplace=True)

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# print the shape of your dataframe
# reset dataframe index
df.reset_index(drop=True, inplace=True)
df.shape

(103, 3)

In [7]:
# add 2 more columns the he dataframe
latitude = [] # create empty latitude list
longitude = [] # create empty logitude list

# function to get the latitude and logitude for each post code from csv file
def get_lat_lng_from_csv():
    location_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
    for index, row in df.iterrows():
        data = location_coordinates[location_coordinates['Postal Code'] == row['PostalCode']]
        latitude.append(float(data['Latitude']))
        longitude.append(float(data['Longitude']))

In [8]:
# assign the list of latitude and logitude tot he dataframe

# call function to add data to the list
get_lat_lng_from_csv()

df['Latitude'] = latitude
df['Longitude'] = longitude
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
toronto_map = folium.Map(location=[toronto_latitude,toronto_longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False
    ).add_to(toronto_map)

toronto_map

Use K-Mean CLustering Algorithm

In [13]:
k=5
toronto_clustering = df_toronto.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans = KMeans(n_clusters = k, random_state=0).fit(toronto_clustering)
print(dir(kmeans))
print(len(kmeans.labels_))
print(kmeans.labels_)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_test_data', '_estimator_type', '_get_param_names', '_transform', 'algorithm', 'cluster_centers_', 'copy_x', 'fit', 'fit_predict', 'fit_transform', 'get_params', 'inertia_', 'init', 'labels_', 'max_iter', 'n_clusters', 'n_init', 'n_iter_', 'n_jobs', 'precompute_distances', 'predict', 'random_state', 'score', 'set_params', 'tol', 'transform', 'verbose']
39
[0 0 0 0 4 0 0 3 0 1 0 3 4 0 3 4 0 4 2 2 2 2 1 2 3 1 2 3 1 2 3 2 0 0 0 0 0
 0 4]


In [14]:
# add kmeans label to the dataframe
df_toronto['kmeans_clustering_labels'] = kmeans.labels_
df_toronto

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PostalCode,Borough,Neighborhood,Latitude,Longitude,kmeans_clustering_labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1
